In [1]:
import xml.etree.ElementTree as ET

In [16]:
from os import path
xml_path = path.relpath("Data/pages_8_47.xml")

In [5]:
with open(xml_path, encoding='utf8') as file:
    tree = ET.parse(file)
    root = tree.getroot()

In [6]:
root.find('./')

<Element '{http://www.tei-c.org/ns/1.0}teiHeader' at 0x00000266671CFD80>

In [ ]:
root.findall('.//{http://www.tei-c.org/ns/1.0}lb')

In [ ]:
root.findall('.//{http://www.tei-c.org/ns/1.0}hi')

In [10]:
from bs4 import BeautifulSoup
with open(xml_path, encoding='utf8') as file:
    contents = file.read()
    BS_contents = BeautifulSoup(contents, 'xml')
    
b_text = BS_contents.find_all('ab')
print(b_text)

[<ab facs="#facs_8_tr_1" type="">
<lb facs="#facs_8_tr_1_tl_1" n="N001"/>I. Wegweiser.
               <lb facs="#facs_8_tr_1_tl_2" n="N002"/>Orientirung in Wien.
               <lb facs="#facs_8_tr_1_tl_3" n="N003"/>Rundgang um die Bastei.
               <lb facs="#facs_8_tr_1_tl_4" n="N004"/>Wenn es für jeden Reisenden unerläßlich ist, sich
               <lb facs="#facs_8_tr_1_tl_5" n="N005"/>in einer fremden Stadt zu orlentiren, so ist es we=
               <lb facs="#facs_8_tr_1_tl_6" n="N006"/>nigstens in Wien sehr leicht. Der <hi>Stephansplatz</hi>
<lb facs="#facs_8_tr_1_tl_7" n="N007"/>llegt so ziemlich mitten in der Stadt, und mit Hilfe
               <lb facs="#facs_8_tr_1_tl_8" n="N008"/>des Planes wird man bald die Hauptpulsader der
               <lb facs="#facs_8_tr_1_tl_9" n="N009"/>innern Stadt kennen lernen; diese führt nämlich vom
               <lb facs="#facs_8_tr_1_tl_10" n="N010"/>Burgthor durch die Burg über den <hi>Michaels=</hi>
<lb facs="#facs_8_tr_1_tl_11" n="

In [5]:
words = root.find('.//{http://www.tei-c.org/ns/1.0}lb').tail

In [119]:
print(words)

I. Wegweiser.
               


In [11]:
text = []
for lb in root.findall('.//{http://www.tei-c.org/ns/1.0}lb'):
    #for hi in root.findall('.//{http://www.tei-c.org/ns/1.0}hi'):
        line = lb.tail
        #if line is None:
            #text.append(
        #else: 
        text.append(line)

Problemanalyse: Extrahiert man nur den Tail, kann man die gehighlighteten Keywords nicht einfangen - und das zusammenfügen funktioniert auch nicht, weil sobald hinter lb irgendetwas steht, kein None gelistet wird und dementsprechend das Keyword zumindest nicht automatisiert eingefügt werden kann (zumindest nicht im Text selbst)- 

Darum: erst mal nur die Keywords aus dem xml extrahieren - der rest vom text wird einfach als txt importiert! 

In [12]:
keywords = []
for hi in root.findall('.//{http://www.tei-c.org/ns/1.0}hi'):
    words = hi.text
    keywords.append(words)

Die Keywords mal behalten, könnten noch interessant werden.

In [17]:
import re

text_path = path.relpath("Data/pages_8_47.txt")
with open(text_path, encoding='utf8') as file:
    text = file.read()
    

In [18]:
text = ' '.join(text.split())

-> issues to resolve: 
- Worttrennungen am Ende der Zeile auflösen
- bei allen anderen = durch - ersetzen
- große J durch I ersetzen (z.B. Jnstitut)
- große I mitten im Wort durch kleine i ersetzen (sind doch kleine Ls, also gehts leider nicht :()

In [19]:
text_clean = re.sub(r'(\w)= ([^uo])',r'\g<1>\g<2>', text)

In [20]:
text_clean = re.sub(r'=', r'-', text_clean)

In [21]:
text_clean = re.sub(r'\bJ(?=[^aoeiuä])', r'I', text_clean)
#to avoid substituting big J in Josef or Jägerzeile e.g.
#re.findall('I\w*', text_clean)

In [22]:
text_clean = re.sub(r'\*+\)', r'', text_clean)

In [23]:
print(text_clean[:500])
path = path.relpath("Data/text_clean.txt")
with open(path, 'w', encoding='utf8') as file:
    file.write(text_clean)
    file.close()

I. Wegweiser. Orientirung in Wien. Rundgang um die Bastei. Wenn es für jeden Reisenden unerläßlich ist, sich in einer fremden Stadt zu orientiren, so ist es wenigstens in Wien sehr leicht. Der Stephansplatz liegt so ziemlich mitten in der Stadt, und mit Hilfe des Planes wird man bald die Hauptpulsader der innern Stadt kennen lernen; diese führt nämlich vom Burgthor durch die Burg über den Michaelsplatz, Kohlmarkt, Graben, Stephansplatz, Bischofs- und Rothenthurmstraße zum Rothenthurmthore, und b


Jetzt: NER ausprobieren, eine Überlegung wäre bspw., alle genannten Orte aus dem Text zu extrahieren

In [18]:
import spacy
nlp = spacy.load("de_core_news_sm")

In [19]:
from spacy.language import Language
@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == ";":
            doc[token.i + 1].is_sent_start = False
            return doc

Noch was einfügen, um die Fußnoten rauszubekommen! -> das hängt aber auch mit dem tagging zusammen. 

In [20]:
custom_nlp = spacy.load("de_core_news_sm")
custom_nlp.add_pipe("set_custom_boundaries", before="parser")
intro_to_nlp = custom_nlp(text_clean)
sentences = list(intro_to_nlp.sents)
for sentence in sentences:
    print(sentence)

I. Wegweiser.
Orientirung in Wien.
Rundgang um die Bastei.
Wenn es für jeden Reisenden unerläßlich ist, sich in einer fremden Stadt zu orlentiren, so ist es wenigstens in Wien sehr leicht.
Der Stephansplatz llegt so ziemlich mitten in der Stadt, und mit Hilfe des Planes wird man bald die Hauptpulsader der innern Stadt kennen lernen; diese führt nämlich vom Burgthor durch die Burg über den Michaelsplatz, Kohlmarkt, Graben, Stephansplatz, Blschofs- und Rothenthurmstraße zum Rothenthurmthore, und bildet die Figur einer doppelten Stufe, deren innerer Winkel St. Stephan ist.
Zu einer dieser Straßen, auf elnen dieser Plätze wird man sich bald finden oder erfragen.
— Eine Bestelgung des Stephansthurmes gibt wohl eine vollständige Uebersicht der Stadt und Umgebung, aber zur Orientirung ist sie keineswegs so wesentlich.
Was die Vorstädte anbelangt, so hat man, aus ihnen auf das Glacis hervorkommend, überall den Stephansthurm vor Augen, wird also leicht in eine bestimmte Gegend der Stadt zurück 

Some simple number-checking

In [67]:
cleaned = ([word for word in intro_to_nlp if not (word.is_stop or word.is_punct or word.is_digit)])

In [102]:
lemma_list = []
for word in cleaned: 
    lemma = word.lemma_
    lemma_list.append(lemma)    
print(lemma_list)

['I.', 'Wegweiser', 'Orientirung', 'Wien', 'Rundgang', 'Bastei', 'reisend', 'unerläßlich', 'fremd', 'Stadt', 'orlentiren', 'Wien', 'Stephansplatz', 'llegen', 'ziemlich', 'mitten', 'Stadt', 'Hilfe', 'Plan', 'Hauptpulsader', 'innern', 'Stadt', 'kennen', 'lernen', 'führen', 'nämlich', 'Burgthor', 'Burg', 'Michaelsplatz', 'Kohlmarkt', 'Graben', 'Stephansplatz', 'Blschof', 'Rothenthurmstraße', 'Rothenthurmthore', 'bilden', 'Figur', 'doppelt', 'Stufe', 'innerer', 'Winkel', 'St.', 'Stephan', 'Straße', 'elnen', 'Platz', 'finden', 'erfragen', 'Bestelgung', 'Stephansthurm', 'vollständig', 'Uebersicht', 'Stadt', 'Umgebung', 'Orientirung', 'keineswegs', 'wesentlich', 'Vorstadt', 'anbelangen', 'Glacis', 'hervorkommend', 'überall', 'Stephansthurm', 'Auge', 'bestimmt', 'Gegend', 'Stadt', 'finden', 'Vorstadt', 'kennen', 'lernen', 'Theil', 'sogar', 'Orientirung', 'Stadt', 'eignen', 'Rundgang', 'Bastei', 'angenehm', 'Spaziergang', 'höchstens', 'anderthalb', 'Stunde', 'Rundgang', 'Reisende', 'sobald', 'b

In [69]:
number_lemma = Counter(lemma_list)
print(number_lemma)

Counter({'stehen': 48, 'rc': 36, 'Wien': 31, 'führen': 30, 'rechts': 29, 'schön': 28, 'enthalten': 27, 'Stadt': 23, 'Kirche': 23, 'links': 22, 'sehen': 21, 'fahren': 20, 'Stunde': 19, 'Gebäude': 19, 'Sammlung': 19, 'hinab': 17, 'kömmen': 16, 'berühmt': 16, 'Garten': 15, 'Haus': 13, 'Vorstadt': 12, 'k.': 12, 'kaiserlich': 12, 'weiterhin': 12, 'befinden': 12, 'reich': 12, 'Burg': 11, 'hinaus': 11, 'Weg': 11, 'besehen': 11, 'Laxenburg': 11, 'finden': 10, 'Wall': 10, 'Palais': 10, 'Karl': 10, 'Baden': 10, 'Park': 10, 'Briel': 10, 'bilden': 9, 'Straße': 9, 'Platz': 9, 'folgend': 9, 'Erzherzog': 9, 'Anstalt': 8, 'interessant': 8, 'Brücke': 8, 'hinauf': 8, 'hoch': 8, 'Linie': 8, 'Liechtenstein': 8, 'Führer': 8, 'Mödling': 8, 'Kaiser': 7, 'erbaut': 7, 'Fürst': 7, 'Institut': 7, 'vorüber': 7, 'Thore': 7, 'erhalten': 7, 'besuchen': 7, 'Nachmittag': 7, 'aufstellen': 7, 'Gasthaus': 7, 'halb': 7, 'Band': 7, 'Glacis': 6, 'Theil': 6, 'gewöhnlich': 6, 'Fischer': 6, 'Bau': 6, 'Höhe': 6, 'oberer': 6, 'H

Erster einfacher Versuch mit Locations: 

In [21]:
mentioned_locations = []
for ent in intro_to_nlp.ents:    
    if ent.label_ in ['LOC']:
        mentioned_locations.append(ent.text)  

In [22]:
print(mentioned_locations)

['Wien', 'Wien', 'Stephansplatz', 'Stadt', 'Planes', 'Burgthor', 'Kohlmarkt', 'Graben', 'Stephansplatz', 'St. Stephan', 'Stephansthurm', 'Stadt', 'höchstens', 'Rasenparterre', 'Gebäudes', 'Erlach', 'Palais des Fürsten Auersperg', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Wien', 'Karlskirche', 'Glaswänden', 'Walle', 'Burgplatze', 'Volksgartens', 'Burggartens', 'Remy', 'Flügelsalon', 'Garten', 'Kärnthnerthor-Theater', 'Kärnthnerstraße', 'Wien', 'Wieden', 'Baden', 'Karlskirche', 'Wienerwaldes', 'Schneeberg', 'Unks', 'Karlskirche', 'Schwarzenberg', 'Raaber Eisenbahn', 'Schwarzenberglschen Palais', 'Walle', 'Artillerie-Haupt-Zeugamt', 'Stadt', 'Weihburggasse', 'Mineralwässer-Trinkkur-Anstalt', 'Rennweg', 'Navelin', 'Herzogs von Koburg', 'Walles', 'Stubenthor', 'Stelnbrücke', 'Wien', 'Landstraße', 'St. Marx', 'Ungarn', 'Neustädter Kanal', 'Walle', 'Walle', 'Hauptmauth', 'Walle', 'Ravelin', 'Donaukanal', 'Kahlengebirge', 'Leopoldstadt', 'Taborstraße', 'Tabor', 'Böhmen', 'D

Offensichtlich werden auch Worte wie Planes oder höchstens als Location erkannt - da muss also noch was getan werden. 
Außerdem: Wien und Stadt (alleine) kann raus. 
- Anderes Problem: die Bastei ist ja eigentlich auch ein Ort - vielleicht werden viele wichtige Orte einfach nicht erkannt? 
- anscheinend werden auch einige Lemma nicht als Ort getaggt
- Außerdem können ja auch Nounphrases wie "die kaiserlichen Stallungen" eine Location sein (das wär ja schon iwie wichtig, wenn ich einen Weg nachzeichnen will) 
- Und: was macht man mit Aufzählungen wie Bischofs- (...)straße? 

In [26]:
mentioned_locations_clean = [ loc for loc in mentioned_locations  if loc != 'Wien' and loc != 'Stadt']

In [27]:
print(mentioned_locations_clean)

['Stephansplatz', 'Planes', 'Burgthor', 'Kohlmarkt', 'Graben', 'Stephansplatz', 'St. Stephan', 'Stephansthurm', 'höchstens', 'Rasenparterre', 'Gebäudes', 'Erlach', 'Palais des Fürsten Auersperg', 'Mariahilfer Hauptstraße', 'Schönbrunn', 'Grenadierkaserne', 'Karlskirche', 'Glaswänden', 'Walle', 'Burgplatze', 'Volksgartens', 'Burggartens', 'Remy', 'Flügelsalon', 'Garten', 'Kärnthnerthor-Theater', 'Kärnthnerstraße', 'Wieden', 'Baden', 'Karlskirche', 'Wienerwaldes', 'Schneeberg', 'Unks', 'Karlskirche', 'Schwarzenberg', 'Raaber Eisenbahn', 'Schwarzenberglschen Palais', 'Walle', 'Artillerie-Haupt-Zeugamt', 'Weihburggasse', 'Mineralwässer-Trinkkur-Anstalt', 'Rennweg', 'Navelin', 'Herzogs von Koburg', 'Walles', 'Stubenthor', 'Stelnbrücke', 'Landstraße', 'St. Marx', 'Ungarn', 'Neustädter Kanal', 'Walle', 'Walle', 'Hauptmauth', 'Walle', 'Ravelin', 'Donaukanal', 'Kahlengebirge', 'Leopoldstadt', 'Taborstraße', 'Tabor', 'Böhmen', 'Donau', 'Franzensbrücke', 'Weißgärbern', 'Erdberg', 'Stadt 10', 'Mül

In [36]:
number_of_loc = {}
for loc in mentioned_locations_clean:
    if not loc in number_of_loc:
        number_of_loc[loc] = 1
    else: 
        number_of_loc[loc] += 1  
print(number_of_loc)          

Trying to lemmatize the text in order to get better results when fetching locations:

In [109]:
cleaned_1 = ([word for word in intro_to_nlp if not (word.is_punct or word.is_digit)])
lemma_list_1 = []
for word in cleaned_1: 
    lemma = word.lemma_
    lemma_list_1.append(lemma)

lemma_text = ' '.join(str(word) for word in lemma_list_1)
nlp_lemma = nlp(lemma_text)    

lemma_locations = []
for word in nlp_lemma.ents:    
    if word.label_ in ['LOC']:
        lemma_locations.append(word.text) 
print(lemma_locations)   

['Wien', 'Stadt', 'Wien', 'Stadt', 'Burgthor', 'Kohlmarkt Graben Stephansplatz Blschof und Rothenthurmstraße', 'Rothenthurmthore', 'St. Stephan', 'Stephansthurm', 'Stephansthurm', 'Stadt', 'Rasenparterre', 'Erlach', 'Fürst Auersperg', 'Mariahilfer', 'Schönbrunn', 'Wien', 'Kettensteg', 'Weldet', 'Hofbibliothek-Gebäud', 'Burgplatze', 'Remy', 'Flügelsalon', 'Garten', 'Spitalplatze', 'Kärnthnerstraße', 'Wien', 'Wiede', 'Baden', 'Wienerwald', 'Schneeberg', 'Bahnhofe', 'Raaber', 'Paals', 'Artillerie-Haupt-Zeugamt', 'Blechthürmch', 'Stadt', 'Mineralwässer-Trinkkur-Anstalt', 'Rennweg', 'Navelin', 'Herzog von', 'Walles', 'Stelnbrücke', 'Wien', 'Landstraße', 'St. Marx', 'Ungarn', 'Neustädter Kanal', 'Pappel', 'Kanalbassin', 'Walle', 'Ravelin', 'Donaukanal', 'Kahlengebirge', 'Leopoldstadt', 'Rothenthurm-Thore', 'Taborstraße', 'Tabor', 'Böhmen', 'Donau', 'Erdberg', 'Laurenzerhof', 'Müllerische Palai', 'Leopoldstadt', 'Karls-Kettenbrücke', 'Schanzl', 'Straße in der Stadt', 'Stadt', 'Militär-Arrest'

Das ist aber auch irgendwie keine so gute Idee, weils hier so auch einfach die Namen verändert - Palais wird zu Palai etc, die hohe Brücke zur hoch Brücke...

Eine Möglichkeit: eigene Lemma-Regeln definieren. Auf der anderen Seite ist das hauptsächlich für die Statistik wichtig, fürs eigentliche Projekt eher nebensächlich... Man könnte auch andere Pakete wie nltk noch ausprobieren.

##########################################

In [45]:
nouns = []
for token in intro_to_nlp:
    if token.pos_ == "PROPN" or token.pos_ == "NOUN":
        nouns.append(token)            

In [50]:
nouns_str  = [str(token) for token in nouns]
print(nouns_str)

['Wegweiser', 'Orientirung', 'Wien', 'Rundgang', 'Bastei', 'Reisenden', 'Stadt', 'Wien', 'Stephansplatz', 'Stadt', 'Hilfe', 'Planes', 'Hauptpulsader', 'Stadt', 'Burgthor', 'Burg', 'Michaelsplatz', 'Kohlmarkt', 'Graben', 'Stephansplatz', 'Rothenthurmstraße', 'Rothenthurmthore', 'Figur', 'Stufe', 'Winkel', 'St.', 'Stephan', 'Straßen', 'Plätze', 'Bestelgung', 'Stephansthurmes', 'Uebersicht', 'Stadt', 'Umgebung', 'Orientirung', 'Vorstädte', 'Glacis', 'Stephansthurm', 'Augen', 'Gegend', 'Stadt', 'Vorstädte', 'Theil', 'Orientirung', 'Stadt', 'Rundgang', 'Bastei', 'Spaziergang', 'Stunden', 'Rundgang', 'Reisende', 'Morgenstunden', 'Besichtigung', 'Sehenswürdigkeiten', 'Folgenden', 'Tag', 'Woche', 'Sonntag', 'Vormittag', 'Stunden', 'Anstalt', 'k.', 'Burg', '*', 'selben', 'Burgplatz', 'Wege', 'Rasenparterre', 'Ecken', 'Wege', 'Wall', 'Burgthor', 'Kaiser', 'Franz', 'I.', 'Nobile', 'Kaisers', 'Staatsanstalten', 'Prädikat', 'k.', 'k.', 'Folgenden', 'Kürze', 'Wahlspruch', 'Justitia', 'Regnorum', 'Fu

In [51]:
from collections import Counter
number_nouns = Counter(nouns_str)

In [52]:
print(number_nouns)

Counter({'Wien': 25, 'Stadt': 23, 'rc': 22, 'Kirche': 22, 'Uhr': 19, 'Gebäude': 17, 'Garten': 14, '*': 13, 'Palais': 13, 'Burg': 11, 'Baden': 11, 'Sammlungen': 11, 'Laxenburg': 11, 'Karl': 10, 'Vorstadt': 10, 'Stunde': 10, 'Stunden': 9, 'Briel': 9, 'Brücke': 8, 'Linie': 8, 'Gasse': 8, 'Liechtenstein': 8, 'Zeit': 8, 'Sammlung': 8, 'Führer': 8, 'Mödling': 8, 'Erzherzogs': 7, 'Walle': 7, 'Straße': 7, 'Thore': 7, 'Weg': 7, 'Glacis': 6, 'Theil': 6, 'Fürsten': 6, 'Platz': 6, 'Institut': 6, 'Bau': 6, 'Hauptmauth': 6, 'Wiens': 6, 'Bastei': 5, 'Tag': 5, 'Vormittag': 5, 'Kaisers': 5, 'Fischer': 5, 'Erlach': 5, 'Höhe': 5, 'Belvedere': 5, 'Navelin': 5, 'Reihe': 5, 'Donau': 5, 'Style': 5, 'Saal': 5, 'Park': 5, 'kais': 5, 'Hofbibliothek': 5, 'Rundgang': 4, 'Stephansplatz': 4, 'St.': 4, 'Anstalt': 4, 'Wege': 4, 'Wall': 4, 'Säulen': 4, 'Schönbrunn': 4, 'Karlskirche': 4, 'Thurm': 4, 'Länge': 4, 'Bahnhofe': 4, 'Eisenbahn': 4, 'Kloster': 4, 'Ravelin': 4, 'Thor': 4, 'Seite': 4, 'Döbling': 4, 'Haus': 4, 'A